In [48]:
import os
import sys

nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [49]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from src.utils import logfile_enabled
from src.dense_net import SimpleDenseNet
from src.config import Config

# from google.colab import userdata
# from huggingface_hub import login
from datasets import load_dataset

# HF_TOKEN = userdata.get("HF_TOKEN")
# login(token=HF_TOKEN)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

DATASET_NAME = "AdityaMayukhSom/MixSub-Hallucinated-Highlight-Features"
TRAIN_COLS = ["MTP", "AVGTP", "MDVTP", "MMDVP"]

ds = load_dataset(DATASET_NAME)

df_train = ds["train"].to_pandas()
df_test = ds["test"].to_pandas()


# X_train = df_train.iloc[:, 0:-1]
# Y_train = df_train.iloc[:, -1]

# X_test = df_test.iloc[:, 0:-1]
# Y_test = df_test.iloc[:, -1]

df = pd.concat([df_train, df_test], axis=0)
X, Y = df.iloc[:, 0:-1], df.iloc[:, -1]

X = X[TRAIN_COLS]
# X_train_features, X_test_features = X_train[training_cols], X_test[training_cols]


X = pd.DataFrame(X, columns=TRAIN_COLS)
Y = Y.astype(int)

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, train_size=0.6, random_state=42
)

ob = StandardScaler()
# only fit with the training data
X_train = ob.fit_transform(X_train)

# do not fit on testing data, only transform
X_test = ob.transform(X_test)

In [13]:
print(Y_train.value_counts())
print(Y_test.value_counts())

IsHallucinated
0    710
1    690
Name: count, dtype: int64
IsHallucinated
1    310
0    290
Name: count, dtype: int64


In [ ]:
sns.pairplot(X)
plt.show()

### Training Logistic Regression


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

lr_clf = LogisticRegression(verbose=1)
lr_clf.fit(X_test, Y_test)

Y_pred_train = lr_clf.predict(X_train)
Y_pred_test = lr_clf.predict(X_test)

accuracy_train = accuracy_score(Y_train, Y_pred_train)
accuracy_test = accuracy_score(Y_test, Y_pred_test)

print(f"Train Accuracy: {accuracy_train * 100:.2f}%")
print(f"Test Accuracy: {accuracy_test * 100:.2f}%")

print("\n\n")

print(classification_report(Y_train, Y_pred_train))
print(classification_report(Y_test, Y_pred_test))

### Training Naive Bayes


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

# Gaussian Naive Bayes
gnb_clf = GaussianNB()
gnb_clf.fit(X_train, Y_train)

# Bernoulli Naive Bayes
bnb_clf = BernoulliNB()
bnb_clf.fit(X_train, Y_train)

# Multinomial Naive Bayes
# mnb_clf = MultinomialNB()
# mnb_clf.fit(X_train, Y_train)

# Change x_clf.predict according to which NB we want to test on.
Y_pred_train = bnb_clf.predict(X_train)
Y_pred_test = bnb_clf.predict(X_test)

accuracy_train = accuracy_score(Y_train, Y_pred_train)
accuracy_test = accuracy_score(Y_test, Y_pred_test)

print(f"Train Accuracy: {accuracy_train * 100:.2f}%")
print(f"Test Accuracy: {accuracy_test * 100:.2f}%")

print("\n\n")

print(classification_report(Y_train, Y_pred_train))
print(classification_report(Y_test, Y_pred_test))

### Training KNN


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

knn_clf = KNeighborsClassifier(n_neighbors=7)
knn_clf.fit(X_train, Y_train)

Y_pred_train = knn_clf.predict(X_train)
Y_pred_test = knn_clf.predict(X_test)

accuracy_train = accuracy_score(Y_train, Y_pred_train)
accuracy_test = accuracy_score(Y_test, Y_pred_test)

print(f"Train Accuracy: {accuracy_train * 100:.2f}%")
print(f"Test Accuracy: {accuracy_test * 100:.2f}%")

print("\n\n")

print(classification_report(Y_train, Y_pred_train))
print(classification_report(Y_test, Y_pred_test))

### Training Random Forest


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=50, random_state=42, max_depth=8)
rf_clf.fit(X_train, Y_train)

Y_pred_train = rf_clf.predict(X_train)
Y_pred_test = rf_clf.predict(X_test)

accuracy_train = accuracy_score(Y_train, Y_pred_train)
accuracy_test = accuracy_score(Y_test, Y_pred_test)

print(f"Train Accuracy: {accuracy_train * 100:.2f}%")
print(f"Test Accuracy: {accuracy_test * 100:.2f}%")

print("\n\n")

print(classification_report(Y_train, Y_pred_train))
print(classification_report(Y_test, Y_pred_test))

### Training Extended Gradient Boost (XGBoost)


In [ ]:
import xgboost as xgb

learning_rate_range = np.arange(0.01, 1, 0.03)
test_XG = []
train_XG = []
for lr in learning_rate_range:
    xgb_classifier = xgb.XGBClassifier(eta=lr)
    xgb_classifier.fit(X_train, Y_train)
    train_XG.append(xgb_classifier.score(X_train, Y_train))
    test_XG.append(xgb_classifier.score(X_test, Y_test))

accuracy_train = accuracy_score(Y_train, Y_pred_train)
accuracy_test = accuracy_score(Y_test, Y_pred_test)

print(f"Train Accuracy: {accuracy_train * 100:.2f}%")
print(f"Test Accuracy: {accuracy_test * 100:.2f}%")

print("\n\n")

print(classification_report(Y_train, Y_pred_train))
print(classification_report(Y_test, Y_pred_test))

In [ ]:
fig = plt.figure(figsize=(10, 7))
plt.plot(learning_rate_range, train_XG, c="orange", label="Train")
plt.plot(learning_rate_range, test_XG, c="m", label="Test")
plt.xlabel("Learning rate")
plt.xticks(learning_rate_range)
plt.ylabel("Accuracy score")
plt.ylim(0.6, 1)
plt.legend(prop={"size": 12}, loc=3)
plt.title("Accuracy score vs. Learning rate of XGBoost", size=14)
plt.show()

In [ ]:
# new learning rate range
learning_rate_range = np.arange(0.01, 0.5, 0.05)
fig = plt.figure(figsize=(19, 17))
idx = 1
# grid search for min_child_weight
for weight in np.arange(0, 4.5, 0.5):
    train = []
    test = []
    for lr in learning_rate_range:
        xgb_classifier = xgb.XGBClassifier(
            eta=lr, reg_lambda=1, min_child_weight=weight
        )
        xgb_classifier.fit(X_train, Y_train)
        train.append(xgb_classifier.score(X_train, Y_train))
        test.append(xgb_classifier.score(X_test, Y_test))
    fig.add_subplot(3, 3, idx)
    idx += 1
    plt.plot(learning_rate_range, train, c="orange", label="Training")
    plt.plot(learning_rate_range, test, c="m", label="Testing")
    plt.xlabel("Learning rate")
    plt.xticks(learning_rate_range)
    plt.ylabel("Accuracy score")
    plt.ylim(0.6, 1)
    plt.legend(prop={"size": 12}, loc=3)
    title = "Min child weight:" + str(weight)
    plt.title(title, size=16)
plt.show()

In [ ]:
log_odds = lr_clf.coef_[0]
odds = np.exp(lr_clf.coef_[0])
lr_features_log = {k: v for k, v in zip(X_train.keys(), log_odds)}
lr_features_no_log = {k: v for k, v in zip(X_train.keys(), odds)}

print("log", lr_features_log)
print("no_log", lr_features_no_log)

### Training SVC


In [ ]:
from sklearn.svm import SVC

svc_clf = SVC(kernel="rbf", degree=7)

svc_clf.fit(X_train, Y_train)

Y_pred_train = svc_clf.predict(X_train)
Y_pred_test = svc_clf.predict(X_test)

accuracy_train = accuracy_score(Y_train, Y_pred_train)
accuracy_test = accuracy_score(Y_test, Y_pred_test)

print(f"Train Accuracy: {accuracy_train * 100:.2f}%")
print(f"Test Accuracy: {accuracy_test * 100:.2f}%")

print("\n\n")

print(classification_report(Y_train, Y_pred_train))
print(classification_report(Y_test, Y_pred_test))

### Training Simple Dense Net Classifier


In [51]:
import torch
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    roc_auc_score,
    precision_recall_curve,
    auc,
)


def compute_metrics(
    model: SimpleDenseNet,
    input_tensor: torch.Tensor,
    true_labels: torch.Tensor,
):
    with torch.no_grad():
        outputs = model(input_tensor)
        predicted_probs = torch.sigmoid(outputs).cpu().numpy()
        predicted = (outputs > 0.5).float().cpu().numpy()

        true_labels = true_labels.cpu().numpy()

        acc = accuracy_score(true_labels, predicted)
        precision = precision_score(true_labels, predicted)
        recall = recall_score(true_labels, predicted)
        f1 = f1_score(true_labels, predicted)

        precision_negative = precision_score(true_labels, predicted, pos_label=0)
        recall_negative = recall_score(true_labels, predicted, pos_label=0)
        f1_negative = f1_score(true_labels, predicted, pos_label=0)

        tn, fp, fn, tp = confusion_matrix(true_labels, predicted).ravel()
        roc_auc = roc_auc_score(true_labels, predicted_probs)

        P, R, thre = precision_recall_curve(true_labels, predicted, pos_label=1)
        pr_auc = auc(R, P)

        roc_auc_negative = roc_auc_score(
            true_labels, 1 - predicted_probs
        )  # If predicted_probs is the probability of the positive class
        P_neg, R_neg, _ = precision_recall_curve(true_labels, predicted, pos_label=0)
        pr_auc_negative = auc(R_neg, P_neg)

        return {
            "Accuracy": acc,
            "Precision": precision,
            "Recall": recall,
            "F1": f1,
            "TP": tp,
            "TN": tn,
            "FP": fp,
            "FN": fn,
            "ROC AUC": roc_auc,
            "PR AUC": pr_auc,
            "Precision-Negative": precision_negative,
            "Recall-Negative": recall_negative,
            "F1-Negative": f1_negative,
            "ROC AUC-Negative": roc_auc_negative,
            "PR AUC-Negative": pr_auc_negative,
        }

#### Code for training the Dense Model and getting the result of all metrics corresponding to the Testing Set.


In [52]:
def compute_accuracy(model, input_tensor: torch.Tensor, true_labels: torch.Tensor):
    with torch.no_grad():
        outputs = model(input_tensor)
        predicted = (outputs > 0.5).float()
        correct = (predicted == true_labels).float().sum()
        accuracy = correct / len(true_labels)
        return accuracy.item()


@logfile_enabled(prefix="simple-dense-net-train")
def train_simple_dense_net(
    model: SimpleDenseNet,
    X_train: torch.Tensor,
    Y_train: torch.Tensor,
    num_epochs: int = 2000,
    print_every: int = 10,
):
    model.train()

    # Define loss and optimizer
    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=0.001,
    )

    # bestValAcc = 0

    for epoch in range(num_epochs):
        optimizer.zero_grad()

        outputs = model(X_train)
        loss: torch.Tensor = criterion(outputs, Y_train)
        loss.backward()

        optimizer.step()

        # Compute training accuracy
        train_accuracy = compute_accuracy(model, X_train, Y_train)

        # Uncomment this if you want to see how the accuracy of testing improves
        # during the training process.
        # X_val_tensor = torch.tensor(X_val_features, dtype=torch.float32).to(device)
        # Y_val_tensor = torch.tensor(Y_val, dtype=torch.float32).view(-1, 1).to(device)

        # val_accuracy = compute_accuracy(denseModel, X_val_tensor, Y_val_tensor)

        # if bestValAcc < val_accuracy:
        #     bestValAcc = val_accuracy
        #     print(f'Saving model with best validation accuracy ...')
        #     torch.save(denseModel.state_dict(), 'llama-' + task + '-best-model')

        if (epoch + 1) % print_every == 0:
            print(
                "Epoch [%5i/%i] :: Loss: %.4f Training Accuracy: %.4f"
                % (epoch + 1, num_epochs, loss.item(), train_accuracy),
            )

#### Compute the metrics using the model on the Test Set


In [ ]:
simple_dense_net = SimpleDenseNet(
    input_dim=4,
    hidden_dim=2048,
).to(Config.DEVICE)

X_train_float64 = X_train.to_numpy(dtype=np.float64)
Y_train_float64 = Y_train.to_numpy(dtype=np.float64)

X_train_tensor = torch.Tensor(X_train_float64).to(Config.DEVICE)
Y_train_tensor = torch.Tensor(Y_train_float64).view(-1, 1).to(Config.DEVICE)

# print(X_train_tensor.shape, Y_train_tensor.shape)

train_simple_dense_net(simple_dense_net, X_train_tensor, Y_train_tensor)

sdn_scripted = torch.jit.script(simple_dense_net)

with (
    open("../data/extractor/dense-model-trained-scripted.pt", "w") as sf,
    open("../data/extractor/dense-model-trained.pt", "w") as f,
):
    sdn_scripted.save(sf)
    torch.save(simple_dense_net.state_dict(), f)


In [ ]:
# sets the model in evaluation loop
simple_dense_net.eval()

X_test_tensor = torch.Tensor(X_test.to_numpy(dtype=np.float64)).to(Config.DEVICE)
Y_test_tensor = (
    torch.Tensor(Y_test.to_numpy(dtype=np.float64)).view(-1, 1).to(Config.DEVICE)
)

# test_metrics = compute_metrics(denseModel, X_train_tensor, Y_train_tensor)
test_metrics = compute_metrics(simple_dense_net, X_test_tensor, Y_test_tensor)

for metric in test_metrics:
    print(f"Testing - {metric}: {test_metrics[metric]:.4f}")